In [2]:
#!/usr/bin/env python3
import cv2
import numpy as np

def skeletonize(binary_img):
    """
    Convert a binary image to its skeleton using a morphological approach.
    Any nonzero pixels in `binary_img` are considered foreground (1), and 
    zero pixels are considered background.
    """
    # Ensure the image is binary (0 or 255)
    # If the image is not already binary, we can threshold it again
    _, bw = cv2.threshold(binary_img, 127, 255, cv2.THRESH_BINARY)
    
    # Prepare an empty image to store the skeleton
    skel = np.zeros_like(bw)
    
    # Get a cross-shaped structuring element
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    
    # Repeat until there's no foreground left to process
    while True:
        # Erode the image
        eroded = cv2.erode(bw, element)
        # Dilate the eroded image
        temp = cv2.dilate(eroded, element)
        # Subtract from original to get the edge
        temp = cv2.subtract(bw, temp)
        # Combine edge with the skeleton
        skel = cv2.bitwise_or(skel, temp)
        # Update bw to be eroded for next iteration
        bw = eroded.copy()

        # Check if the image has been completely eroded
        if cv2.countNonZero(bw) == 0:
            break

    return skel


def main():
    # 1. Read the image
    # Replace 'thumb.jpg' with the path to your image
    image = cv2.imread('2927418551_f442e6b0f5_b.jpg')
    if image is None:
        print("Could not open or find the image. Check the file path.")
        return

    # 2. Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 3. Optionally enhance the contrast using histogram equalization
    # (This can help make ridges more distinct)
    enhanced = cv2.equalizeHist(gray)

    # 4. (Optional) Smooth the image slightly to remove noise
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)

    # 5. Threshold (Binarize) the image
    #    You can choose between Otsu or adaptive threshold depending on lighting
    # -- Otsu's Thresholding --
    # _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # -- Adaptive Thresholding --
    # (Helps if the image has non-uniform lighting)
    binary = cv2.adaptiveThreshold(
        blurred,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        35,  # blockSize - adjust for image resolution
        15   # constant subtracted from mean
    )

    # 6. Morphological closing to fill small holes in ridges
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # 7. Skeletonize (thin the ridges to one-pixel thickness)
    skeleton = skeletonize(closed)

    # 8. Show results
    # Original
    cv2.imshow('Original Image', image)
    # Enhanced grayscale
    cv2.imshow('Enhanced Grayscale', enhanced)
    # Binary / thresholded
    cv2.imshow('Binary Image', binary)
    # Closed (morphological operation)
    cv2.imshow('Closed Image', closed)
    # Final Skeleton
    cv2.imshow('Skeleton (Thinned Ridges)', skeleton)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # # 9. (Optional) Save the resulting skeleton image
    # cv2.imwrite('fingerprint_skeleton.jpg', skeleton)
    # print("Skeleton image saved as 'fingerprint_skeleton.jpg'.")

if __name__ == "__main__":
    main()
